In [7]:
import pandas as pd

df = pd.read_csv("student_response.csv")

class_data = pd.read_csv("class_data.csv")

student_preferences = pd.read_csv("student_preferences.csv")


In [8]:
names = df['Name']
a_like = list(df['attendance_likelihood'])
head_volunteer = df['head_volunteer']

df = df.drop(columns=['Name', 'attendance_likelihood', 'head_volunteer',
       'stay_all_day', 'event', 'specific_event', 'event.1',
       'specific_event.1', 'Which do you prefer?    =====================>',
       'Event grader', 'Proctor', 'Runner/Floater',
        'Would you be interested in being a photographer? \r\n\r\n0: No\r\n1: Yes'])
df.columns

Index(['Time_1', 'Time_2', 'Time_3', 'Time_4', 'Time_5', 'Time_6', 'Time_7',
       'Time_8'],
      dtype='object')

In [9]:
a_like_convert = {'Decently likely': 1.0, 
                  'YES!': 1.0}

for i, val in enumerate(a_like):
    if val in a_like_convert:
        a_like[i] = a_like_convert[val]
    else:
        a_like[i] = 0.6

#this could be used for robust, although would be impractical

def create_scenarios(att_like):

    if len(att_like) == 0:
        return [(1, [])]

    results = []

    next_res = create_scenarios(att_like[1:])

    for scen in next_res:
        results.append((scen[0] * att_like[0], [1] + scen[1]))
        if not att_like[0] == 1:
            results.append((scen[0] * (1-att_like[0]), [0] + scen[1]))

    return results

scenarios = create_scenarios(a_like)

In [3]:
ind_to_event = {}
event_to_ind = {}

ind_to_name = {}
name_to_ind = {}

for i, name in enumerate(names):
    ind_to_name[i] = name
    name_to_ind[name] = i

for i, event in enumerate(class_data['Event']):
    ind_to_event[i] = event
    event_to_ind[event] = i

In [4]:
s_pref = {}

cols = student_preferences.columns[1:]

for i, row in student_preferences.iterrows():
    name = row['Name']
    
    name_ind = name_to_ind[name]
    
    class_inds = [index for index, value in enumerate(row[1:]) if value != 0]
    class_names = [event_to_ind[cols[ind]] for ind in class_inds]

    s_pref[name_ind] = class_names




In [71]:
import gurobipy as grb

# Create a new model
model = grb.Model("example_lp")
model.ModelSense = grb.GRB.MINIMIZE

num_students = df.shape[0]
num_times = 8
num_classes = 32

# Dictionary to store decision variables

#vars[(i, j, k, 1)] = 1 if student i is a principal volunteer for class k at time j
#vars[(i, j, k, 0)] = 1 if student i is a non-principal volunteer for class k at time j

vars = {}

vars_to_minimize_availability_slack = []

for i in range(num_students):
    for j in range(num_times):
        if not df.iloc[i, j]:
            # If volunteer not available, set variables to 0 and add constraints
            for k in range(num_classes):
                vars[(i, j, k, 0)] = model.addVar(name=f"x_{i}_{j}_{k}_np", vtype=grb.GRB.BINARY)
                vars[(i, j, k, 1)] = model.addVar(name=f"x_{i}_{j}_{k}_p", vtype=grb.GRB.BINARY)
                vars_to_minimize_availability_slack.append(model.addVar(name=f"avail_{i}_{j}_k_1"))
                model.addConstr(vars[(i, j, k, 1)] == vars_to_minimize_availability_slack[-1], name=f"no_volunteer_p_{i}_{j}_{k}")
                vars_to_minimize_availability_slack.append(model.addVar(name=f"avail_{i}_{j}_k_0"))
                model.addConstr(vars[(i, j, k, 0)] == vars_to_minimize_availability_slack[-1], name=f"no_volunteer_np_{i}_{j}_{k}")
            continue

        for k in range(num_classes):
            # Add decision variables
            vars[(i, j, k, 0)] = model.addVar(name=f"x_{i}_{j}_{k}_np", vtype=grb.GRB.BINARY)
            vars[(i, j, k, 1)] = model.addVar(name=f"x_{i}_{j}_{k}_p", vtype=grb.GRB.BINARY)

            # A person can either be a principal or non-principal volunteer
            model.addConstr(
                vars[(i, j, k, 0)] + vars[(i, j, k, 1)] <= 1, 
                name=f"princ_or_nonprinc_{i}_{j}_{k}"
            )

            # If not a head volunteer, they cannot be a principal volunteer
            if head_volunteer[i] != 1:
                model.addConstr(
                    vars[(i, j, k, 1)] == 0, 
                    name=f"not_head_volunteer_{i}_{j}_{k}"
                )

# A person can only volunteer for one class at a time
for i in range(num_students):
    for j in range(num_times):
        model.addConstr(
            sum(vars[(i, j, k, p)] for k in range(num_classes) for p in [0, 1]) <= 1,
            name=f"one_class_{i}_{j}"
        )

vars_to_minimize_slack_p = []
vars_to_minimize_slack = []

vars_to_minimize_constr = []

# Ensure the demand for each event is not exceeded
for j in range(num_times):
    for k in range(num_classes):
        vars_to_minimize_slack.append(model.addVar(name=f"demand_slack_{j}_{k}", lb=0))
        model.addConstr(
            sum(vars[(i, j, k, p)] for i in range(num_students) for p in [0, 1]) >= int(class_data['needed_volunteers'][k]) - vars_to_minimize_slack[-1],
            name=f"meet_demand_{j}_{k}"
        )

        # one principal volunteer per event
        vars_to_minimize_slack_p.append(model.addVar(name=f"demand_slack_{j}_{k}_p", lb=0))
        
        model.addConstr(
            sum(vars[(i, j, k, 1)] for i in range(num_students)) + vars_to_minimize_slack_p[-1] == 1,
            name=f"meet_p_demand_{j}_{k}"
        ) 


# Cost to meet demand as closely as possible
# minimize 

# minimize |vars[(i, j, k_1, p)] - vars[(i, j+1, k_2, p)]| for all i, j, p, k_1 neq k_2

vars_to_minimize = []

vars_to_minimize_constr = []

for i in range(num_students):
    for j in range(num_times-1):
        for k in range(num_classes):
                for p in [0, 1]:
                    vars_to_minimize_constr.append(vars[(i, j, k, p)] - vars[(i, j+1, k, p)])
                    vars_to_minimize.append(model.addVar(name=f"minimize_{i}_{j}_{k}_{p}", lb=-grb.GRB.INFINITY))

                    model.addConstr(vars_to_minimize[-1] >= vars_to_minimize_constr[-1])
                    model.addConstr(vars_to_minimize[-1] >= -vars_to_minimize_constr[-1])                 

vars_to_minimize_pref = []

# Add cost to assign students to thier preferred events
for i in range(num_students):
    student_pref = s_pref[i]
    for j in range(num_times):
        for k in range(num_classes):
            if k in student_pref:
                continue
            for p in [0,1]:
                vars_to_minimize_pref.append((vars[i,j,k,p]))


model.setObjectiveN(sum(vars_to_minimize_availability_slack), 1)
model.setObjectiveN(sum(vars_to_minimize_slack_p), 2, 1)
model.setObjectiveN(sum(vars_to_minimize), 2, 1)
model.setObjectiveN(sum(vars_to_minimize_pref), 3)
model.setObjectiveN(sum(vars_to_minimize_slack), 0)

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == grb.GRB.OPTIMAL:
    print("Optimal solution found.")
    print(f"Objective value: {model.objVal}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1250U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 126944 rows, 110912 columns and 443424 nonzeros
Model fingerprint: 0x5e760f68
Variable types: 63808 continuous, 47104 integer (47104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 58880 rows and 30752 columns
Presolve time: 0.89s
Presolved: 6

In [ ]:
sumX = 0
for var in vars_to_minimize:
        sumX += var.x
        if var.x > 0:
                continue
                print(var.VarName)

print(sumX)

312.0


In [29]:
print(len(vars_to_minimize_slack))

512


In [27]:
print(vars_to_minimize_slack[-1].X
      )

1.0
